In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [21]:
from scipy import stats
from scipy.stats import norm

from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import pickle

import warnings
warnings.filterwarnings('ignore')


In [22]:
# Customize matplotlib default settings
matplotlib.rcParams.update({'font.size': 16})
plt.rcParams["figure.figsize"] = (20,10)

In [23]:
# plotly
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

## 1. Get stock data

In [24]:
!pip install alpha_vantage

In [9]:
# from alpha_vantage.timeseries import TimeSeries

In [10]:
# ts = TimeSeries(key='YOUR_API_KEY', output_format='pandas')
# ts = TimeSeries(key='', output_format='pandas')

In [11]:
# Get the daily stock data and the call's metadata
# stock_data, stock_meta_data = ts.get_daily('GOOGL', outputsize='full')

In [12]:
# stock_meta_data

{'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
 '2. Symbol': 'GOOGL',
 '3. Last Refreshed': '2019-04-17 16:00:01',
 '4. Output Size': 'Full size',
 '5. Time Zone': 'US/Eastern'}

Output of the above:  
{'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
 '2. Symbol': 'GOOGL',
 '3. Last Refreshed': '2019-04-17 16:00:01',
 '4. Output Size': 'Full size',
 '5. Time Zone': 'US/Eastern'}

In [13]:
# stock_data.head()

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0


In [14]:
# rename columns so that they only have varname-acceptible characters in them
# stock_data.columns = ["open", "high", "low", "close", "volume"]

In [19]:
# pickle the data 

# myoutfile = "google_stock_data.pkl"
# with open(myoutfile, 'wb') as picklefile:
#     pickle.dump(stock_data, picklefile)

In [16]:
# unpickle
with open("google_stock_data.pkl", 'rb') as picklefile: 
    stock_data = pickle.load(picklefile)

stock_data.head()

,open,high,low,close,volume
date,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0


In [17]:
stock_data.tail()

,open,high,low,close,volume
date,,,,,
2019-04-11,1208.90,1212.74,1204.54,1209.59,849947.0
2019-04-12,1215.62,1223.05,1213.29,1222.73,1215610.0
2019-04-15,1224.09,1229.30,1214.56,1226.53,1189974.0
2019-04-16,1230.00,1235.98,1225.04,1231.91,1127963.0
2019-04-17,1237.00,1245.06,1232.90,1240.14,1518286.0


## 2. Data exploration

In [ ]:
stock_data.head()

In [ ]:
stock_data.describe()

In [ ]:
print("Date range:", stock_data.index.min(), "to", stock_data.index.max())

In [ ]:
# datarow with the max closing price
stock_data[stock_data.close == stock_data.close.max()]

In [ ]:
# datarow with the max volume
stock_data[stock_data.volume == stock_data.volume.max()]

In [1]:
trace = go.Scatter(
                    x = stock_data.index,
                    y = stock_data.close,
                    mode = "lines",
                    name = "daily close prices",
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                    text= stock_data.index)

layout = dict(title = 'GOOGL Stock Daily Close Prices',
              xaxis= dict(title= 'Date', zeroline= False),
              yaxis= dict(title= 'Close price')
             )
fig = dict(data = [trace], layout = layout)
iplot(fig)

NameError: name 'go' is not defined

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout